In [1]:
!pip install tensorflow
!pip install keras

In [36]:
#librerías, no es necesario volverlas a importar

import pandas as pd
import os

from sklearn.metrics import classification_report

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, Sequential

%run ../0_Data/Util.ipynb

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 1. Datos

In [5]:
#Dataset de entranamiento
df_train = pd.read_csv(os.path.join('..','0_Data','dataset','train.csv'),index_col='id', encoding='utf8')

#Dataset de test
df_test = pd.read_csv(os.path.join('..','0_Data','dataset','test.csv'),index_col='id', encoding='utf8')

In [22]:
x_train_original = df_train['text']
x_test_original = df_test['text']
train_y  = df_train['target'].values

In [20]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 200 # max number of words in a question to use


In [42]:


## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train_original))
train_X = tokenizer.texts_to_sequences(x_train_original)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)

model = Sequential()
model.add(layers.Embedding(input_dim=max_features, 
                           output_dim=embed_size, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

model.fit(train_X, train_y,
                    epochs=6,
                    batch_size=512)


#model.fit(train_X, train_y, batch_size=512, epochs=2)
loss, accuracy = model.evaluate(train_X, train_y, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 300)          15000000  
_________________________________________________________________
flatten_10 (Flatten)         (None, 60000)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                600010    
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 15,600,021
Trainable params: 15,600,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
15/15 [==============================] - 3s 198ms/step - loss: 0.6786 - accuracy: 0.5774
Epoch 2/6
15/15 [==============================] - 3s 193ms/step - loss: 0.5605 - accuracy: 0.7261
Epoch 3/6
15/15 [========================

In [53]:
test_X = tokenizer.texts_to_sequences(x_test_original)

## Pad the sentences 
test_X = pad_sequences(test_X, maxlen=maxlen)


#y_test = model.predict(test_X, batch_size=1024, verbose=1)
y_test = model.predict_classes(test_X, batch_size=1024, verbose=1)

4/4 [==============================] - 0s 35ms/step


In [44]:
pred_y = model.predict_classes(train_X, batch_size=1024, verbose=1)

8/8 [==============================] - 0s 48ms/step


In [45]:
print("Report: \n{}".format(classification_report(train_y, pred_y)))

Report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4342
           1       1.00      0.98      0.99      3271

    accuracy                           0.99      7613
   macro avg       0.99      0.99      0.99      7613
weighted avg       0.99      0.99      0.99      7613



In [46]:
pred_y

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [47]:
submission = pd.read_csv(os.path.join('..','0_Data','dataset','sample_submission.csv'))

In [56]:
len(y_test)

3263

In [54]:
submission['target'] = y_test

In [55]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [57]:
submission.to_csv("tensor_embeddings.csv", index=False)